### Preamble

In [ ]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [ ]:
# PIP INSTALLS

# for data handling:
#! pip install pandas

# for reading Duration entries (WIP, unused yet):
#! pip install text2digits

# for imputing lat/long:
#! pip install geopy

In [ ]:
# IMPORTS

import re
import pandas as pd

from numpy import select
from functools import partial
from text2digits import text2digits
from datetime import datetime,timedelta
from geopy.geocoders import Nominatim#,OpenCage
from geopy.extra.rate_limiter import RateLimiter


# initialise things:
ua = \
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
k = \
"cc6fe73ae2a64a7c937b8f8553dce0a1"

Geogetter = partial(Nominatim(user_agent=ua).geocode , language='en')
#Geogetter = partial(OpenCage(k).geocode , language='en')
Geoget = RateLimiter(Geogetter , min_delay_seconds=1)

Reader = text2digits.Text2Digits()

In [ ]:
# CONSTANTS

statesUSA = {
    'AL' : 'Alabama',
    'AK' : 'Alaska',
    'AS' : 'American Samoa',
    'AZ' : 'Arizona',
    'AR' : 'Arkansas',
    'CA' : 'California',
    'CO' : 'Colorado',
    'CT' : 'Connecticut',
    'DE' : 'Delaware',
    'DC' : 'District of Columbia',
    'FL' : 'Florida',
    'GA' : 'Georgia',
    'GU' : 'Guam',
    'HI' : 'Hawaii',
    'ID' : 'Idaho',
    'IL' : 'Illinois',
    'IN' : 'Indiana',
    'IA' : 'Iowa',
    'KS' : 'Kansas',
    'KY' : 'Kentucky',
    'LA' : 'Louisiana',
    'ME' : 'Maine',
    'MD' : 'Maryland',
    'MA' : 'Massachusetts',
    'MI' : 'Michigan',
    'MN' : 'Minnesota',
    'MS' : 'Mississippi',
    'MO' : 'Missouri',
    'MT' : 'Montana',
    'NE' : 'Nebraska',
    'NV' : 'Nevada',
    'NH' : 'New Hampshire',
    'NJ' : 'New Jersey',
    'NM' : 'New Mexico',
    'NY' : 'New York',
    'NC' : 'North Carolina',
    'ND' : 'North Dakota',
    'OH' : 'Ohio',
    'OK' : 'Oklahoma',
    'OR' : 'Oregon',
    'PA' : 'Pennsylvania',
    'PR' : 'Puerto Rico',
    'RI' : 'Rhode Island',
    'SC' : 'South Carolina',
    'SD' : 'South Dakota',
    'TN' : 'Tennessee',
    'TX' : 'Texas',
    'UT' : 'Utah',
    'VT' : 'Vermont',
    'VA' : 'Virginia',
    'WA' : 'Washington',
    'WV' : 'West Virginia',
    'WI' : 'Wisconsin',
    'WY' : 'Wyoming'
}

statesCND = {
  'AB' : 'Alberta',
  'BC' : 'British Columbia',
  'MB' : 'Manitoba',
  'NB' : 'New Brunswick',
  'NL' : 'Newfoundland and Labrador',
  'NS' : 'Nova Scotia',
  'NT' : 'Northwest Territories',
  'NU' : 'Nunavut',
  'ON' : 'Ontario',
  'PE' : 'Prince Edward Island',
  'QC' : 'Quebec',
  'SK' : 'Saskatchewan',
  'YT' : 'Yukon'
}

allStates = {**statesUSA, **statesCND}

shapeMapping = {"light"     : "Flash",
                "circle"    : "Orb",
                "teardrop"  : "Teardrop",
                "cigar"     : "Cylinder",
                "disk"      : "Disc",
                "unknown"   : "Indiscernible",
                "oval"      : "Egg",
                "other"     : "Indiscernible",
                "sphere"    : "Orb",
                "changing"  : "Metamorphing",
                "formation" : "Swarm",
                "flash"     : "Flash",
                "chevron"   : "Chevron",
                "0"         : "Egg",
                "triangle"  : "Triangle",
                "fireball"  : "Flash",
                "cross"     : "Cross",
                "rectangle" : "Rectangle",
                "diamond"   : "Diamond",
                "cylinder"  : "Cylinder",
                "egg"       : "Egg",
                "cone"      : "Cone"}

regexes = {
    1      : r'(?<![A-z])se?c?o?n?d?s?',
    2      : r'(?<![A-z])m(?:in)?(?:ute)?s?',
    3      : r'(?<![A-z])ho?u?r?s?',
    'time' : r'\d{1,2}\s?[:.;]\s?(?:\d{1,2}\s?[:.;]\s?)?\d\d(?=\D|\b)',
    'vals' : r'\.?\d+ ?[.,]? ?\d*'
}

### Functions

In [ ]:
# PRIVATE

def _findnth(string , substring , n):
    """ Find n-th occurance """
    # https://stackoverflow.com/a/13094326/12955879
    if n == 1:
        return string.find(substring)
    else:
        return string.find( substring , _findnth(string,substring,n-1)+1 )

    
def _sieve(dictionary):
    """ Return key(s) of non-empty dictionary entry """
    return [k for k,v in dictionary.items() if v]


def _datetimeToSeconds(datetimeObj):
    return _timediff([datetime(1900,1,1) , datetimeObj]).total_seconds()


def _timediff(window):
    """ Difference of two datetime objects, accounting for spans across midnight """
    # can be done as (end + 24 - start) % 24 but this is simpler/more readable.
    if window[0] <= window[1]:
        return window[1] - window[0]
    else:
        return window[1] + timedelta(days=1) - window[0]


def _findSingleton(timescales):
    return {
        i+1 : 60**i for i in range(3)
    }[_sieve(timescales)]


def _findTime(times , scales=dict()):
    sievedScales = _sieve(scales)
    
    disqualifiers = [
        not sievedScales and len(times) == 1,
        len(times) > 2,
        len(sievedScales) > len(times)
    ]
    if any(disqualifiers):  return None
    
    print(sievedScales)
    window = list()
    if not sievedScales:
        for time,pos in times:
            form = '%H:%M:%S' if time.count(':')==1 else '%M:%S'
            window.append( datetime.strptime(time,form) )
    elif len(sievedScales) == 1:
        form = {
            3 : '%H:%M:%S',
            2 : '%M:%S:%f',
            1 : '%S:%f'                 # assuming we don't get any '00:00:05 seconds'? Damn... I'm not gonna bother.
        }[int( *sievedScales )]
        for time,pos in times:
            if (time.count(':') == 2) and (1 in sievedScales):
                pass
            chop = slice(5) if time.count(':')==1 else slice(8)
            window.append( datetime.strptime(time,form[chop]) )
    elif len(sievedScales) == 2:
        # WIP
        pass

    if len(window) == 2:
        span = _timediff(window).total_seconds()
        if any(map( lambda time:time[0].count(':')==1 , times )):
            window[-1] = window[0] + timedelta(seconds=span/2)          # average if any 2-parters
        else:
            return span                                                 # difference for 3-parters only
    
    return _datetimeToSeconds(window[-1])
    
def _extractDurationData(text):
    raw = text
    
    # catches 2- or 3-part clock formats:
    times = list()
    for time in re.findall(regexes['time'] , text):
        text = text.replace(time , ' ' , 1)
        clean = time.replace(' ','').replace(';',':').replace('.',':')
        clean = ':'.join( part.zfill(2) for part in clean.split(':') )
        times.append( (clean , raw.find(time)) )
        
    # catches leftover numerics:
    values = list()
    for value in re.findall(regexes['vals'] , text):
        text = text.replace(value , ' ' , 1)
        clean = value.replace(' ','').replace(',','.')
        values.append( (clean , raw.find(value)) )
    
    # catches leftover timescale words:
    scales = {
        1 : list(),
        2 : list(),
        3 : list()
    }
    for sc in scales:
        for i,match in enumerate( re.findall(regexes[sc],text) ):
            text = text.replace(match , ' ' , 1)
            scales[sc].append( _findnth(raw,match,i+1) )
    
    return times,values,scales

    
def _countryName(geo):
    name = geo.address.split(', ')[-1]
    # Nomatim returns 'United States of America', so fix that:
    return name if 'United States' not in name else 'United States'


def _assignGeo(m,city,abbrev):
    """ Acts as a switch/case for imputing geographical data. """
    # columns are, in order: Country - Latitude - Longitude
    # 3 gaps - all 3 need imputation
    # 2 gaps - lat/long need imputation (they always come together)
    # 1 gap  - only Country needs imputation
    address = city
    if abbrev in allStates.keys():
        address += ", " + allStates[abbrev]
    
    geo = Geoget(address)
    
    if not geo:
        print(city)
        return (None,None,None)
    else:
        return {
            3 : ( _countryName(geo) , geo.latitude , geo.longitude ),
            2 : ( None , geo.latitude , geo.longitude ),
            1 : ( _countryName(geo) , None , None )
        }[m]

In [ ]:
# TEST CELL FOR READING DURATION DATA

tests = list()
tests.append('05:00hrs to 06:00hrs')
tests.append('0.05m')
tests.append('30:00 to 40:00')
tests.append('15 sec')
tests.append('2h to 3h')
tests.append('1 min 30s')
tests.append('06:30 to 08:00hrs')
tests.append('1and 1/2 minutes')
tests.append('15s to 1 min')
tests.append('1/2 hr')
tests.append('For 5-10 Min')

test = pd.Series(tests)

test = cleanDuration(test)

for text in test:
    # COMBINATIONS:
    #  singleton:
    #   only 3-part time       - timediff from Time field (ignore for now)
    #   only 2-part time       - DURATION (as mm:ss) / ignore?
    #   only value             - ignore
    #   only scale             - assume 1 ('hour' is 01:00:00)
    #  pair:
    #   2x 3-times             - average (assume leftmost is hours)
    #   2x 2-times             - average (assume leftmost is minutes)
    #   value-scale pair       - DURATION
    #   2/3-time & scale       - DURATION with leftmost part as scale (e.g. '6:45hrs' is 06:45:00, '02:30m' is 00:02:30)
    #  triplet:
    #   value-value and scale  - average, with scale applying to both (e.g. '3-5 mins' is 00:04:00)
    #   value-scale-value      - fuse with scale as first value (e.g. '1m30' is 00:01:30)
    #   2x 2/3-times & scale   - timediff, with scale applying to both ('6:15-6:45hrs' is 00:30:00)
    #  quadruplet:
    #   2x value-scale pairs decreasing   - fuse (e.g. '2 mins 30 sec' is 00:02:30)
    #   2x value-scale pairs monotone     - average (e.g. '30 sec - 2 mins' is 00:01:15, '4 min - 5 min' is 00:04:30)
    #   2x 2/3-time and 2x scales         - timediff with scale as leftmost part (e.g. '30:00m - 1:00hr')
    #   value-scale and time-scale        - average in pairs (e.g. '30s to 01:00m' is 00:00:45)
    dur = None
    
    times,values,scales = _extractDurationData(text)
    
    print("text:",text)
    print("times:",times)
    print("values:",values)
    print("scales:",scales)

    scaleCounts = {
        3 : len(scales[3]),           # hour
        2 : len(scales[2]),           # minute
        1 : len(scales[1])            # second
    }
    componentCounts = {
        'times'  : len(times),
        'values' : len(values),
        'scale'  : sum(scaleCounts.values())
    }
    totalCount = sum(componentCounts.values())
    print("components:",componentCounts)
    print("timescales:",scaleCounts)
    
    # S       - assume 1                 'hour'
    # T S     - scale is leftmost time   '05:00 mins'
    # V S     - scale is value           '5hrs'
    # T T
    #   2T 2T  - mean with MM:SS            '03:00 to 03:30'
    #   2T 3T  - ignore
    #   3T 2T  - ignore
    #   3T 3T  - diff with HH:MM:SS         '22:30:00 to 01:00:00'
    # T T S   - scale is leftmost time   '03:00 to 03:45 mins'
    #   config irrelevant
    # V V S   - scale is both values     '3-5 hrs'
    # V S V   - scale is first value     '1 min 30'
    # rest are combinations of these.
    #
    if totalCount == 1 and componentCounts['scale'] == 1:
        dur = _findSingleton(scales)
    elif totalCount == 2:
        if componentCounts['times'] == 2:
            dur = _findTime(times)
        elif componentCounts['times'] == 1 and componentCounts['scale'] == 1:
            dur = _findTime(times,scales)
        elif componentCounts['values'] == 1 and componentCounts['scale'] == 1:
            pass
        
    print("duration (s):",dur)
    print()
    #return dur

In [ ]:
# PUBLIC

def cleanDuration(serie):
    # preserve floats and homogenises case:
    serie = serie.astype(str)
    serie = serie.str.lower()
    
    # common fractions made decimal, so entries like '5/6 hours' can be read as 05:30:00
    serie = serie.str.replace(r'(?:and|&)\s?(?:a)?' , ' ')
    serie = serie.str.replace(r'half' , '1/2')
    serie = serie.str.replace(r'(?<!\d)\s*1[\/\\]4(?!\d)' , '.25')
    serie = serie.str.replace(r'(?<!\d)\s*1[\/\\]3(?!\d)' , '.33')
    serie = serie.str.replace(r'(?<!\d)\s*1[\/\\]2(?!\d)' , '.5')
    
    return serie


def readDuration(text):
    """ Using regex procedures to remedy the godless decision to enter duration as freetext. """
    # Each Duration entry is made of any number of COMPONENTS:
    # TIMES
    #  2-part:
    #   00:45      (could be hh:mm or mm:ss)
    #   6:45       (could be h:mm or m:ss)
    #  3-part:
    #   08:15:25   (hh:mm:ss)
    # 
    # VALUES
    #  5           (leave as-is)
    #  3.3         (treat as literal, so 3:30)
    #  3.5         (special case, assume 3:30 too)
    #  >= 3.6      (treat as fraction, so >= 3:36)
    # 
    # SCALES
    #  s sec secs second seconds
    #  m min mins minute minutes
    #  h hr hrs hour hours
    # 
    # 
    # 
    # COMBINATIONS:
    #  singleton:
    #   only 3-part time       - timediff from Time field (ignore for now)
    #   only 2-part time       - DURATION (as mm:ss) / ignore?
    #   only value             - ignore
    #   only scale             - assume 1 ('hour' is 01:00:00)
    #  pair:
    #   2x 3-times             - average (assume leftmost is hours)
    #   2x 2-times             - average (assume leftmost is minutes)
    #   two values/scales      - ignore
    #   value-scale pair       - DURATION
    #   2/3-time & scale       - DURATION with leftmost part as scale (e.g. '6:45hrs' is 06:45:00, '02:30m' is 00:02:30)
    #  triplet:
    #   value-value and scale  - average, with scale applying to both (e.g. '3-5 mins' is 00:04:00)
    #   value-scale-value      - fuse with scale as first value (e.g. '1m30' is 00:01:30)
    #   2x 2/3-times & scale   - timediff, with scale applying to both ('6:15-6:45hrs' is 00:30:00)
    #  quadruplet:
    #   2x value-scale pairs decreasing   - fuse (e.g. '2 mins 30 sec' is 00:02:30)
    #   2x value-scale pairs monotone     - average (e.g. '30 sec - 2 mins' is 00:01:15, '4 min - 5 min' is 00:04:30)
    #   2x 2/3-time and 2x scales         - timediff with scale as leftmost part (e.g. '30:00m - 1:00hr')
    #   value-scale and time-scale        - average in pairs (e.g. '30s to 01:00m' is 00:00:45)
    dur = None
    
    times,values,scales = _extractDurationData(text)
    
    scaleCounts = {
        3 : len(scales[3]),           # hour
        2 : len(scales[2]),           # minute
        1 : len(scales[1])            # second
    }
    componentCounts = {
        'times'  : len(times),
        'values' : len(values),
        'scale'  : sum(scaleCounts.values())
    }
    totalCount = sum(componentCounts.values())
    
    # S       - assume 1                 'hour'
    # T S     - scale is leftmost time   '05:00 mins'
    # V S     - scale is value           '5hrs'
    # T T
    #   2T 2T  - mean with MM:SS            '03:00 to 03:30'
    #   2T 3T  - ignore
    #   3T 2T  - ignore
    #   3T 3T  - diff with HH:MM:SS         '22:30:00 to 01:00:00'
    # T T S   - scale is leftmost time   '03:00 to 03:45 mins'
    #   config irrelevant
    # V V S   - scale is both values     '3-5 hrs'
    # V S V   - scale is first value     '1 min 30'
    # rest are combinations of these (?)
    if totalCount == 1 and componentCounts['scale'] == 1:
        dur = _findSingleton(scales)
    elif totalCount == 2:
        if componentCounts['times'] == 2:
            dur = _findTime(times)
        elif componentCounts['times'] == 1 and componentCounts['scale'] == 1:
            dur = _findTime(times,scales)
        elif componentCounts['values'] == 1 and componentCounts['scale'] == 1:
            # WIP
            pass
    
    return dur

    # limitations:
    # yet to figure out how to handle recurring sightings. Anything over the scale of hours is ignored currently.
    # other languages not considered "1 hora" gets nullified
    # number-words are ignored (for now) "twenty minutes" or "an hour"
    # ~1,000 entries use the Duration field as an end time (relative to Time field), so could take difference?
    # order between components is not considered — e.g. '5 minutes 15' is computed same as '5 to 15 minutes'; 00:10:00


def imputeLocal(states):
    """ More accurate country labelling. """
    # For 554 "U.S." entries, a state isn't even given. A lot of these are foreign cities
    #  without even an American town named after them, meaning the label is plainly incorrect.
    #  A few are also Puerto Rican towns, so I included it in the set of (American) States.
    conds = [
        states.isnull(),
        states.isin( set(statesUSA.keys()) ),
        states.isin( set(statesCND.keys()) )
    ]
    choices = [None , 'United States' , 'Canada']        
    return select(conds , choices)


def imputeWeb(missinglist , cities , states):
    # inline is faster apparently, and I need speed.
    overlay = [ _assignGeo(m,city,state) if m>0 else (None,None,None) for m,city,state in zip(missinglist,cities,states) ]
    return pd.DataFrame( overlay , columns=['Country','Latitude','Longitude'] )

### Mains

In [ ]:
if __name__ == '__main__':
    for i in range(1):
        file = 'nuforc_raw'
        DF = pd.read_excel(
            file+'.xlsx',
            parse_dates=[['Date','Time']]
        )
        
        # Time:
        DF.rename(
            columns = {'Date_Time':'Datetime'},
            inplace = True
        )
        
        # Shape:
        DF['Shape'] = DF.Shape.map(shapeMapping).astype('category')

        # Country (impute local, only U.S. or Canada):
        countries_old = DF.Country.copy()
        DF['Country'] = imputeLocal(DF.State)

        # Country, Latitude, Longitude (impute from web):
        if False:
            toImpute = ['Country','Latitude','Longitude']
            missingnos = DF[toImpute].isnull().sum(axis=1).to_list()
            DF[toImpute] = DF[toImpute].combine_first(
                imputeWeb(missingnos , DF.City.values , DF.State.values)
            )
        
        # reinstate original country if no better guess
        DF['Country'] = DF.Country.combine_first(countries_old)

        # Duration:
        if False:
            DF['Duration'] = cleanDuration(DF.Duration)
            DF['Duration'] = DF.Duration.map(
                readDuration,
                na_action='ignore'
            )
            DF = DF.rename({'Duration':'Duration (s)'} , axis=1)

        print(i, "\n" , DF)
        DF.to_excel( file+'_final.xlsx' )